In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Passo 1: Definir Modelos Pydantic para Resultados Estruturados

Estes modelos definem o **esquema** que os agentes irão retornar. Utilizar `response_format` com Pydantic garante:
- ✅ Extração de dados com segurança de tipo
- ✅ Validação automática
- ✅ Sem erros de análise em respostas de texto livre
- ✅ Roteamento condicional fácil com base nos campos


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Passo 2: Criar a Ferramenta de Reserva de Hotel

Esta ferramenta será utilizada pelo **availability_agent** para verificar se há quartos disponíveis. Usamos o decorador `@ai_function` para:
- Converter uma função Python numa ferramenta acessível por IA
- Gerar automaticamente o esquema JSON para o LLM
- Gerir a validação de parâmetros
- Permitir a invocação automática por agentes

Para esta demonstração:
- **Estocolmo, Seattle, Tóquio, Londres, Amesterdão** → Têm quartos disponíveis ✅
- **Todas as outras cidades** → Sem quartos disponíveis ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Passo 3: Definir Funções de Condição para o Encaminhamento

Estas funções analisam a resposta do agente e determinam qual caminho seguir no fluxo de trabalho.

**Padrão Principal:**
1. Verificar se a mensagem é `AgentExecutorResponse`
2. Analisar o output estruturado (modelo Pydantic)
3. Retornar `True` ou `False` para controlar o encaminhamento

O fluxo de trabalho irá avaliar estas condições nos **elos** para decidir qual executor invocar a seguir.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Passo 4: Criar Executor de Exibição Personalizado

Os executores são componentes de fluxo de trabalho que realizam transformações ou efeitos colaterais. Utilizamos o decorador `@executor` para criar um executor personalizado que exibe o resultado final.

**Conceitos-chave:**
- `@executor(id="...")` - Regista uma função como executor de fluxo de trabalho
- `WorkflowContext[Never, str]` - Dicas de tipo para entrada/saída
- `ctx.yield_output(...)` - Produz o resultado final do fluxo de trabalho


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Passo 5: Carregar Variáveis de Ambiente

Configure o cliente LLM. Este exemplo funciona com:
- **Modelos GitHub** (Plano gratuito com token do GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Passo 6: Criar Agentes de IA com Resultados Estruturados

Criamos **três agentes especializados**, cada um envolvido num `AgentExecutor`:

1. **availability_agent** - Verifica a disponibilidade de hotéis utilizando a ferramenta
2. **alternative_agent** - Sugere cidades alternativas (quando não há quartos disponíveis)
3. **booking_agent** - Incentiva a reserva (quando há quartos disponíveis)

**Características Principais:**
- `tools=[hotel_booking]` - Fornece a ferramenta ao agente
- `response_format=PydanticModel` - Garante saída JSON estruturada
- `AgentExecutor(..., id="...")` - Envolve o agente para uso em fluxos de trabalho


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Passo 7: Construir o Fluxo de Trabalho com Ligações Condicionais

Agora usamos `WorkflowBuilder` para construir o gráfico com encaminhamento condicional:

**Estrutura do Fluxo de Trabalho:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Métodos Principais:**
- `.set_start_executor(...)` - Define o ponto de entrada
- `.add_edge(from, to, condition=...)` - Adiciona uma ligação condicional
- `.build()` - Finaliza o fluxo de trabalho


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Passo 8: Executar Caso de Teste 1 - Cidade SEM Disponibilidade (Paris)

Vamos testar o caminho de **sem disponibilidade** ao solicitar hotéis em Paris (que não tem quartos na nossa simulação).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Passo 9: Executar Caso de Teste 2 - Cidade COM Disponibilidade (Estocolmo)

Agora vamos testar o caminho de **disponibilidade** ao solicitar hotéis em Estocolmo (que tem quartos na nossa simulação).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Principais Aprendizagens e Próximos Passos

### ✅ O Que Aprendeste:

1. **Padrão WorkflowBuilder**
   - Usa `.set_start_executor()` para definir o ponto de entrada
   - Usa `.add_edge(from, to, condition=...)` para encaminhamento condicional
   - Chama `.build()` para finalizar o fluxo de trabalho

2. **Encaminhamento Condicional**
   - Funções de condição inspecionam `AgentExecutorResponse`
   - Analisa saídas estruturadas para tomar decisões de encaminhamento
   - Retorna `True` para ativar uma ligação, `False` para ignorá-la

3. **Integração de Ferramentas**
   - Usa `@ai_function` para converter funções Python em ferramentas de IA
   - Agentes chamam ferramentas automaticamente quando necessário
   - Ferramentas retornam JSON que os agentes podem interpretar

4. **Saídas Estruturadas**
   - Usa modelos Pydantic para extração de dados com segurança de tipos
   - Define `response_format=MyModel` ao criar agentes
   - Analisa respostas com `Model.model_validate_json()`

5. **Executores Personalizados**
   - Usa `@executor(id="...")` para criar componentes de fluxo de trabalho
   - Executores podem transformar dados ou realizar efeitos colaterais
   - Usa `ctx.yield_output()` para produzir resultados do fluxo de trabalho

### 🚀 Aplicações no Mundo Real:

- **Reserva de Viagens**: Verificar disponibilidade, sugerir alternativas, comparar opções
- **Serviço ao Cliente**: Encaminhar com base no tipo de problema, sentimento, prioridade
- **E-commerce**: Verificar inventário, sugerir alternativas, processar pedidos
- **Moderação de Conteúdo**: Encaminhar com base em pontuações de toxicidade, sinalizações de utilizadores
- **Fluxos de Aprovação**: Encaminhar com base no montante, função do utilizador, nível de risco
- **Processamento Multi-etapas**: Encaminhar com base na qualidade e completude dos dados

### 📚 Próximos Passos:

- Adicionar condições mais complexas (múltiplos critérios)
- Implementar ciclos com gestão de estado do fluxo de trabalho
- Adicionar subfluxos para componentes reutilizáveis
- Integrar com APIs reais (reserva de hotéis, sistemas de inventário)
- Adicionar tratamento de erros e caminhos de fallback
- Visualizar fluxos de trabalho com as ferramentas de visualização integradas



---

**Aviso**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos pela precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se uma tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
